In [2]:
## Conversion Reference: https://github.com/onnx/sklearn-onnx

# Convert into ONNX format.
from skl2onnx import to_onnx
import joblib
import numpy as np
import onnxruntime as rt
from skl2onnx.common.data_types import FloatTensorType


tree_model_path = "/Users/apple/work/Culling/selection_algo/dumps/random_forest_model_trained2Batches_2Aug.joblib"
DUMP_model_path = "/Users/apple/work/Culling/selection_algo/dumps/to_onnx/sample_selection_model.onnx"

clr = joblib.load(tree_model_path)

FEAT_SIZE = 37

input_feat = np.random.rand(100,FEAT_SIZE).astype(np.float32)
X_test = input_feat[50:60,:]
X_test = np.zeros((2,FEAT_SIZE)).astype(np.float32)

print("X_test ", X_test.shape)

initial_type = [("input_feature", FloatTensorType([None, FEAT_SIZE]))]
## To make probabilities instead of labels, set "target_opset=12"
onx = to_onnx(clr, X_test[:1], initial_types=initial_type, target_opset=12)

# initial_type = [("float_input", FloatTensorType([None, 4]))]
# options = {id(clr): {"raw_scores": True}}
# onx2 = convert_sklearn(
#     clr, initial_types=initial_type, options=options, target_opset=12
# )
              
with open(DUMP_model_path, "wb") as f:
    f.write(onx.SerializeToString())



X_test  (2, 37)


# Inference using onnxruntime


In [3]:
# Inference using onnxruntime

sess = rt.InferenceSession(DUMP_model_path, providers=["CPUExecutionProvider"])
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
print("Input expected type:", sess.get_inputs()[0].type)
print("passed input ", type(X_test))

print("input_name ", input_name)
print("label_name ", label_name)

# pred_onx = sess.run([label_name], {input_name: X_test.astype(np.float32)})
pred_onx = sess.run(None, {input_name: X_test.astype(np.float32)})

print("pred_onx ", pred_onx)

Input expected type: tensor(float)
passed input  <class 'numpy.ndarray'>
input_name  input_feature
label_name  output_label
pred_onx  [array([1, 1], dtype=int64), [{0: 0.1156991720199585, 1: 0.8843008279800415}, {0: 0.1156991720199585, 1: 0.8843008279800415}]]
